Referencing nuget packages

In [1]:
#r "nuget:XPlot.Plotly"
#r "nuget:Deedle"
#r "nuget:DotNetZip"

Installing package XPlot.Plotly...............done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

Installing package Deedle......done!

Successfully added reference to package Deedle, version 2.1.0

Installing package DotNetZip.....done!

Successfully added reference to package DotNetZip, version 1.13.4

**Type in password**

In [2]:
var password = "type password here";

In [29]:
using System.Linq;
using System.Collections.Generic;

(string, int[])[] SingleLineSource(string folder, int lineName) => new[] { (folder, new[] { lineName }), };
(string, int[])[] AllLinesSource() => 
    new[]
    {
        ("BadLines",       new[] { 490, 675, 677, 681, 671 } ),
        ("GoodLines",      new[] { 783, 834, 782, 837, 785 } ),
        ("InnerCityLines", new[] { 403, 505, 216, 906, 302 } ),
    };

**Decide if you want to play with a single line or all of them at once. Use corresponding line of code.**

In [30]:
//var sources = SingleLineSource("GoodLines", 837);
var sources = AllLinesSource();

Main code

In [31]:
using Deedle;
using Ionic.Zip;
using System.IO;
using System;
using System.Globalization;

Frame<int, string> LoadData(string folder, string lineName)
{
    var zip = ZipFile.Read(string.Format(@"Data/Samples/{1}/Departures-{0}.zip", lineName, folder));
    zip.Password = password;
    using (var stream = zip.Entries.First().OpenReader())
    {
        var frame = 
            Frame
                .ReadCsv(stream)
                .Where(kvp => !string.IsNullOrEmpty(kvp.Value.GetAs<string>("ScheduledArrivalTime")));

        display($"Successfully read data: {frame.RowCount} records");

        var sa = frame.GetColumn<string>("ScheduledArrivalTime").Values.ToArray();
        var aa = frame.GetColumn<string>("ArrivalTime").Values.ToArray();
        var deviation = sa.Select((s, i) => (int)(DateTime.Parse(aa[i]) - DateTime.Parse(s)).TotalSeconds).ToArray();
        frame.AddColumn("Deviation", deviation);    

        return frame;
    }
}

In [32]:
public class StatsValues
{
    public double Min      { get; set; }
    public double Max      { get; set; }
    public double Mean     { get; set; }
    public double Median   { get; set; }
    public double StdDev   { get; set; }
    public string Folder   { get; set; }
    public string LineName { get; set; }
}

StatsValues CalcStats(Series<int, int> series, string folder, string lineName) => new StatsValues 
{
    Min      = Stats.min(series),
    Max      = Stats.max(series),
    Mean     = Stats.mean(series),
    StdDev   = Stats.stdDev(series),
    Median   = Stats.median(series),
    Folder   = folder,
    LineName = lineName,
};

In [34]:
var allFrames =
    sources
        .SelectMany(x => x.Item2.Select(ln => new { folder = x.Item1, lineName = ln.ToString() }))
        .ToArray()
        .Select(x => new { frame = LoadData(x.folder, x.lineName), x.folder, x.lineName })
        .ToDictionary(x => x.lineName);
display("All Loaded!");

Successfully read data: 16114 records

Successfully read data: 10756 records

Successfully read data: 27804 records

Successfully read data: 50754 records

Successfully read data: 54641 records

...

Successfully read data: 45695 records

Successfully read data: 507730 records

Successfully read data: 541946 records

Successfully read data: 304044 records

All Loaded!

In [35]:
var allStats = 
    allFrames
        .Values
        .Select(x => CalcStats(x.frame.GetColumn<int>("Deviation"), x.folder, x.lineName))
        .OrderBy(x => x.Mean)
        .ToArray();
display(allStats);
var allStatsFrame = Frame.FromRecords(allStats);

index,Min,Max,Mean,Median,StdDev,Folder,LineName
0,-881,2341,60.06255430060817,89,243.643919776572,BadLines,490
1,-538,3349,66.14143041237114,-34,417.53698831794,InnerCityLines,403
2,-349,1780,70.54125851501699,46,144.65238568710708,GoodLines,837
3,-4448,1887,91.17078456870395,45,224.2982717043588,BadLines,681
4,-2534,7659,103.98105715329572,63,247.38620040520533,InnerCityLines,906
5,-947,5146,106.67408664535397,65,241.4356113285932,InnerCityLines,302
6,-1153,4225,107.02586716270926,57,290.7971000078602,InnerCityLines,505
7,-4722,5029,116.00091921284309,70,199.58899838047952,GoodLines,834
8,-692,1815,137.2573184163237,124,228.1809293259011,GoodLines,783
9,-170,1712,140.37086277426553,105,195.76103318404876,BadLines,675


In [36]:
var folder = "BadLines";
var lineName = "681";

var frame = LoadData(folder, lineName);
//var frame = allFrames[lineName].frame;

Successfully read data: 50754 records

In [37]:
(int, double, double) CalcBuckets(Series<int, int> series)
{
    var stats = CalcStats(series, "", "");
    var bucketsNumber = 100;
    var bucketSize = (double)(stats.Max - stats.Min) / bucketsNumber;

    int getBucket(int value) => (int)(((double)value - stats.Min) / bucketSize);

    var distribution = series.GroupBy(kvp => getBucket(kvp.Value)).Select(kvp => kvp.Value.KeyCount);
    var maxBucket = distribution.Values.Max();    
    
    return (bucketsNumber, bucketSize, maxBucket);
}

Filtering function

In [38]:
static Frame<int, string> Filter(this Frame<int, string> frame, params Func<KeyValuePair<int, ObjectSeries<string>>, bool>[] conditions)
{
    var result = frame;
    foreach (var condition in conditions)
        result = result.Where(condition);

    return result;
}

In [39]:
using XPlot.Plotly;

PlotlyChart MakePlot(Series<int, int> series, string forWhat, int w, int h)
{
    var title = $"Deviation from Schedule (sec) for {forWhat}";
    
    var stats = CalcStats(series, "", "");
    var (bucketsNumber, bucketSize, maxBucket) = CalcBuckets(series);
    
    display(stats);
    
    var hist = 
        new Graph.Histogram
        {
            x = series.Values, 
            xbins   = new Graph.Xbins { start = stats.Min, end = stats.Max, size = bucketSize }, 
            marker  = new Graph.Marker { color = "yellow", line = new Graph.Line { color = "gray", width = 1 }},
            opacity = 0.75, 
            name = "Distribution"
        };

    var zero   = new Graph.Scatter { name = "Zero",   x = new[] { 0, 0 }, y = new[] { 0, maxBucket }, };
    var mean   = new Graph.Scatter { name = "Mean",   x = new[] { stats.Mean, stats.Mean }, y = new[] { 0, maxBucket }, };
    var median = new Graph.Scatter { name = "Median", x = new[] { stats.Median, stats.Median }, y = new[] { 0, maxBucket }, };

    var stdDev = 
        new Graph.Scatter
        {
            x = new[] { stats.Mean-stats.StdDev, stats.Mean-stats.StdDev, stats.Mean+stats.StdDev, stats.Mean+stats.StdDev },
            y = new[] { maxBucket, 0, 0, maxBucket },
            name = "StdDev",
        };

    var traces = new Graph.Trace[] { hist, mean, median, stdDev, zero };

    var plot = Chart.Plot(traces);  
    plot.WithXTitle("Deviation");
    plot.WithYTitle("Numner of arrivals");
    plot.WithTitle(title);
    plot.WithWidth(w);
    plot.WithHeight(h);
    return plot;
}

In [40]:
PlotlyChart CreatePlot(string lineName = null, int? direction = null, int? stopCode = null, int? w = null, int? h = null)
{
    var (frame, folder, lineName2) = 
        lineName != null 
        ? (allFrames[lineName].frame, allFrames[lineName].folder, allFrames[lineName].lineName)
        : (allFrames.First().Value.frame, allFrames.First().Value.folder, allFrames.First().Value.lineName);
    
    var title = $"Line {lineName2} [{folder}]";
    if (direction != null)
    {
        title += $", Direction {direction}";
        frame = frame.Filter(
            kvp => kvp.Value.GetAs<int>("Direction") == direction.Value);
    }
    if (stopCode != null)
    {
        title += $", StopCode {stopCode}";
        frame = frame.Filter(
            kvp => kvp.Value.GetAs<int>("IrsStopCode") == stopCode);
    }
    
    var deviationSeries = frame.GetColumn<int>("Deviation");
    var plot = MakePlot(deviationSeries, title, w ?? 800, h ?? 600);
    return plot;
}


**If no parameters specified, the first loade line will be used**

**You can specify lineName, direction, stopCode for analysis plots**

**To see the list of available stopCodes use the following cell**

In [44]:
display(CreatePlot());
display(CreatePlot("837"));
display(CreatePlot(stopCode: 40866));

Min,Max,Mean,Median,StdDev,Folder,LineName
-881,2341,60.06255430060817,89,243.643919776572,,


Min,Max,Mean,Median,StdDev,Folder,LineName
-349,1780,70.54125851501699,46,144.65238568710708,,


Min,Max,Mean,Median,StdDev,Folder,LineName
-68,2223,172.98847262247838,156,153.54782400866256,,


In [46]:
void DisplayStops(string lineName = null)
{
    var frame = lineName == null ? allFrames.First().Value.frame : allFrames[lineName].frame;
    var stops = frame.GetColumn<string>("IrsStopCode").Values.Distinct().ToArray();
    display(stops);
}

In [48]:
DisplayStops();
//DisplayStops("837");

index,value
0,6831
1,40866
2,40868
3,41032
4,40871
5,40872
6,40877
7,40869
8,40884
9,40885


index,value
0,18629
1,22801
2,3467
3,14718
4,14719
5,14720
6,14721
7,14722
8,14723
9,3701
